In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from haversine import haversine

In [2]:
df_tcd=pd.read_csv('df_tcd.csv', encoding='ANSI')

In [3]:
#전체 정류장 불러오기(부천 +외부)
df_sttn=pd.read_csv('STTN.csv', encoding='ANSI') #통합정거장 데이터
# 통합 정거장 데이터 가져오기 및 열이름 변경
df_sttn=df_sttn[['승차정류장ID','SIG']]
df_sttn.columns=['정류장 ID', '행정동']
df_sttn=df_sttn.drop_duplicates()
df_sttn

,정류장 ID,행정동
0,7046,상동
1,7086,중동
2,7097,중동
3,7206,심곡동
4,7230,심곡동
...,...,...
4929,9213197,구로구
4930,9514077,구로구
4931,9514250,구로구
4932,9606416,양천구


In [7]:
# df_total 데이터에서 정류장의 id 근거로 승차 정류장좌표 columns 생성
df_total=pd.merge(df_tcd, df_sttn, left_on='승차정류장ID', right_on='정류장 ID')

# df_total 데이터에서 정류장의 id 근거로 하차 정류장좌표 columns 생성
df_total=pd.merge(df_total, df_sttn, left_on='하차정류장 ID', right_on='정류장 ID',suffixes=('_승차', '_하차'))
df_total
df_total.to_csv('df_total.csv')

KeyboardInterrupt: 

In [ ]:
# 승하차 시간 datetime 데이터 타입으로 변경
df_total['승차일시']=pd.to_datetime(df_total['승차일시'],format='%Y%m%d%H%M%S')
df_total['하차일시']=pd.to_datetime(df_total['하차일시'],format='%Y%m%d%H%M%S') # dropna  로 결측치 저리 안할시 처리 불가

In [ ]:
df_total=df_total.sort_values(['가상카드번호','승차일시']);

In [ ]:
df_total=df_total.reset_index()
df_total=df_total.drop(['index','Unnamed: 0'], axis=1)

In [ ]:
# 환승인원 구하기
temp_set=set()
for i in range(1,len(df_total)):
    if ((df_total.loc[i,'가상카드번호']==df_total.loc[i-1,'가상카드번호']) & ((df_total.loc[i,'승차일시']- df_total.loc[i-1,'하차일시']).seconds<1800)) & (df_total.loc[i,'노선 ID']!=df_total.loc[i-1,'노선 ID']):
        temp_set.add(i-1)
        temp_set.add(i)

temp_list=list(temp_set)
df_total.iloc[temp_list]

In [ ]:
df_change=df_total.iloc[temp_list]
df_change=df_change.sort_values(['가상카드번호','승차일시']);
df_change
df_change.to_csv('df_change.csv', encoding="ANSI")

In [2]:
df_change=pd.read_csv('df_change.csv', encoding="ANSI")

In [3]:
df_change

,Unnamed: 0,가상카드번호,노선 ID,승차일시,승차정류장ID,하차일시,하차정류장 ID,환승횟수,이용객수(다인승),정류장 ID_승차,행정동_승차,정류장 ID_하차,행정동_하차
0,8,+++QU9IQLcfyK57T+fmxCWG0ya4PbacMkPokT08yA4c=,102,2019-10-28 18:07:23,1806,2019-10-28 18:19:11,1804.0,0,1,1806,부평구,1804,대산동
1,9,+++QU9IQLcfyK57T+fmxCWG0ya4PbacMkPokT08yA4c=,41030037,2019-10-28 18:26:35,4100939,2019-10-28 18:31:01,4100932.0,1,1,4100939,대산동,4100932,대산동
2,19,++//j0S1oE/5mCjtMJ+RYGiibcS5eOudaQzvb/dqsqY=,41056004,2019-10-24 09:09:46,4109242,2019-10-24 09:17:05,4100810.0,0,1,4109242,양천구,4100810,성곡동
3,20,++//j0S1oE/5mCjtMJ+RYGiibcS5eOudaQzvb/dqsqY=,207,2019-10-24 09:19:30,2753,2019-10-24 09:34:38,2758.0,1,1,2753,성곡동,2758,상동
4,21,++//j0S1oE/5mCjtMJ+RYGiibcS5eOudaQzvb/dqsqY=,207,2019-10-24 17:11:36,2758,2019-10-24 17:24:13,2753.0,0,1,2758,상동,2753,성곡동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4294326,11979545,zzzwlavqWHOTKHDWSa+m7Z0eHTnouBxjc1dFy/ErH4Q=,114,2021-04-24 13:55:17,4804,2021-04-24 15:19:02,311.0,1,1,4804,심곡동,311,은평구
4294327,11979547,zzzwlavqWHOTKHDWSa+m7Z0eHTnouBxjc1dFy/ErH4Q=,41030002,2021-04-25 13:39:44,4101108,2021-04-25 14:01:46,4101275.0,0,1,4101108,중동,4101275,심곡동
4294328,11979548,zzzwlavqWHOTKHDWSa+m7Z0eHTnouBxjc1dFy/ErH4Q=,114,2021-04-25 14:06:53,4804,2021-04-25 14:36:19,234.0,1,1,4804,심곡동,234,구로구
4294329,11979549,zzzwlavqWHOTKHDWSa+m7Z0eHTnouBxjc1dFy/ErH4Q=,102,2021-04-25 18:53:42,1806,2021-04-25 19:03:00,1805.0,1,1,1806,부평구,1805,대산동


In [ ]:
df_change['승차일시']=pd.to_datetime(df_change['승차일시'],format='%Y-%m-%d %H:%M:%S')
df_change['하차일시']=pd.to_datetime(df_change['하차일시'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
df_change['환승시간']=np.nan
df_change['환승거리']=np.nan
df_change['구분자']=0

In [9]:
df_change

,Unnamed: 0,가상카드번호,노선 ID,승차일시,승차정류장ID,하차일시,하차정류장 ID,환승횟수,이용객수(다인승),정류장 ID_승차,행정동_승차,정류장 ID_하차,행정동_하차,환승시간,환승거리,구분자
0,8,+++QU9IQLcfyK57T+fmxCWG0ya4PbacMkPokT08yA4c=,102,2019-10-28 18:07:23,1806,2019-10-28 18:19:11,1804.0,0,1,1806,부평구,1804,대산동,NaN,NaN,0
1,9,+++QU9IQLcfyK57T+fmxCWG0ya4PbacMkPokT08yA4c=,41030037,2019-10-28 18:26:35,4100939,2019-10-28 18:31:01,4100932.0,1,1,4100939,대산동,4100932,대산동,NaN,NaN,0
2,19,++//j0S1oE/5mCjtMJ+RYGiibcS5eOudaQzvb/dqsqY=,41056004,2019-10-24 09:09:46,4109242,2019-10-24 09:17:05,4100810.0,0,1,4109242,양천구,4100810,성곡동,NaN,NaN,0
3,20,++//j0S1oE/5mCjtMJ+RYGiibcS5eOudaQzvb/dqsqY=,207,2019-10-24 09:19:30,2753,2019-10-24 09:34:38,2758.0,1,1,2753,성곡동,2758,상동,NaN,NaN,0
4,21,++//j0S1oE/5mCjtMJ+RYGiibcS5eOudaQzvb/dqsqY=,207,2019-10-24 17:11:36,2758,2019-10-24 17:24:13,2753.0,0,1,2758,상동,2753,성곡동,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4294326,11979545,zzzwlavqWHOTKHDWSa+m7Z0eHTnouBxjc1dFy/ErH4Q=,114,2021-04-24 13:55:17,4804,2021-04-24 15:19:02,311.0,1,1,4804,심곡동,311,은평구,NaN,NaN,0
4294327,11979547,zzzwlavqWHOTKHDWSa+m7Z0eHTnouBxjc1dFy/ErH4Q=,41030002,2021-04-25 13:39:44,4101108,2021-04-25 14:01:46,4101275.0,0,1,4101108,중동,4101275,심곡동,NaN,NaN,0
4294328,11979548,zzzwlavqWHOTKHDWSa+m7Z0eHTnouBxjc1dFy/ErH4Q=,114,2021-04-25 14:06:53,4804,2021-04-25 14:36:19,234.0,1,1,4804,심곡동,234,구로구,NaN,NaN,0
4294329,11979549,zzzwlavqWHOTKHDWSa+m7Z0eHTnouBxjc1dFy/ErH4Q=,102,2021-04-25 18:53:42,1806,2021-04-25 19:03:00,1805.0,1,1,1806,부평구,1805,대산동,NaN,NaN,0


In [34]:
cnt=0
for i in range(1,len(df_change)):
    if ((df_change['승차일시'].iloc[i]- df_change['하차일시'].iloc[i-1]).total_seconds()<=1800) & (df_change['가상카드번호'].iloc[i]==df_change['가상카드번호'].iloc[i-1])& (df_change.loc[i,'노선 ID']!=df_change.loc[i-1,'노선 ID']):
        df_change['구분자'].iloc[i]=df_change['구분자'].iloc[i-1]
    else:
        cnt+=1
        df_change['구분자'].iloc[i]=cnt

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [35]:
df_change[(df_change['구분자']==113)]

,Unnamed: 0,Unnamed: 0.1,가상카드번호,노선 ID,승차일시,승차정류장ID,하차일시,하차정류장 ID,환승횟수,이용객수(다인승),정류장 ID_승차,행정동_승차,정류장 ID_하차,행정동_하차,환승시간,환승거리,구분자
230,230,623,++CGxI65qxsgBT5BXiMlw3djALPySZXcnMA3AW0UPsw=,41030017,2020-10-24 07:47:30,4196649,2020-10-24 07:54:18,4178867.0,0,1,4196649,소사본동,4178867,소사본동,NaN,NaN,113


In [36]:
df_change.to_csv('change_구분자1.csv',encoding='ANSI')

In [43]:
df_change=pd.read_csv('change_구분자1.csv',encoding='ANSI')

df_change

In [50]:
df_change=df_change.sort_values(['승차일시']);

In [1]:
df_change['전_가상카드번호']=df_change['가상카드번호'].shift(1)
df_change['전_하차일시']=df_change['하차일시'].shift(1)
df_change['전_환승횟수']=df_change['환승횟수'].shift(1)
df_change

NameError: name 'df_change' is not defined

In [4]:
df_change2=df_change[['구분자',"노선 ID","승차일시","하차일시","승차정류장ID","하차정류장 ID","이용객수(다인승)"]]
df_change2

,구분자,노선 ID,승차일시,하차일시,승차정류장ID,하차정류장 ID,이용객수(다인승)
0,0,102,2019-10-28 18:07:23,2019-10-28 18:19:11,1806,1804.0,1
1,0,41030037,2019-10-28 18:26:35,2019-10-28 18:31:01,4100939,4100932.0,1
2,1,41056004,2019-10-24 09:09:46,2019-10-24 09:17:05,4109242,4100810.0,1
3,1,207,2019-10-24 09:19:30,2019-10-24 09:34:38,2753,2758.0,1
4,2,207,2019-10-24 17:11:36,2019-10-24 17:24:13,2758,2753.0,1
...,...,...,...,...,...,...,...
4294326,2126041,114,2021-04-24 13:55:17,2021-04-24 15:19:02,4804,311.0,1
4294327,2126042,41030002,2021-04-25 13:39:44,2021-04-25 14:01:46,4101108,4101275.0,1
4294328,2126042,114,2021-04-25 14:06:53,2021-04-25 14:36:19,4804,234.0,1
4294329,2126043,102,2021-04-25 18:53:42,2021-04-25 19:03:00,1806,1805.0,1


In [52]:
df_change[(df_change['가상카드번호']=='++CGxI65qxsgBT5BXiMlw3djALPySZXcnMA3AW0UPsw=')]

,Unnamed: 0,Unnamed: 0.1,가상카드번호,노선 ID,승차일시,승차정류장ID,하차일시,하차정류장 ID,환승횟수,이용객수(다인승),정류장 ID_승차,행정동_승차,정류장 ID_하차,행정동_하차,환승시간,환승거리,구분자
220,220,606,++CGxI65qxsgBT5BXiMlw3djALPySZXcnMA3AW0UPsw=,41015001,2019-10-23 07:32:52,4196605,2019-10-23 07:34:35,4102119.0,0,1,4196605,소사본동,4102119,범안동,NaN,NaN,108
221,221,607,++CGxI65qxsgBT5BXiMlw3djALPySZXcnMA3AW0UPsw=,41072002,2019-10-23 07:38:47,4113587,2019-10-23 07:42:20,4116112.0,1,1,4113587,범안동,4116112,범안동,NaN,NaN,108
222,222,608,++CGxI65qxsgBT5BXiMlw3djALPySZXcnMA3AW0UPsw=,41015001,2019-10-24 07:25:31,4196605,2019-10-24 07:26:40,4102119.0,0,1,4196605,소사본동,4102119,범안동,NaN,NaN,109
223,223,609,++CGxI65qxsgBT5BXiMlw3djALPySZXcnMA3AW0UPsw=,41030037,2019-10-24 07:31:18,4113587,2019-10-24 07:34:58,4116112.0,1,1,4113587,범안동,4116112,범안동,NaN,NaN,109
224,224,611,++CGxI65qxsgBT5BXiMlw3djALPySZXcnMA3AW0UPsw=,41015001,2019-10-25 07:05:27,4196605,2019-10-25 07:09:12,4102119.0,0,1,4196605,소사본동,4102119,범안동,NaN,NaN,110
225,225,612,++CGxI65qxsgBT5BXiMlw3djALPySZXcnMA3AW0UPsw=,41072002,2019-10-25 07:13:12,4113587,2019-10-25 07:18:01,4116112.0,1,1,4113587,범안동,4116112,범안동,NaN,NaN,110
226,226,618,++CGxI65qxsgBT5BXiMlw3djALPySZXcnMA3AW0UPsw=,41015001,2019-10-28 07:05:23,4196605,2019-10-28 07:06:40,4102119.0,0,1,4196605,소사본동,4102119,범안동,NaN,NaN,111
227,227,619,++CGxI65qxsgBT5BXiMlw3djALPySZXcnMA3AW0UPsw=,41072002,2019-10-28 07:16:11,4113587,2019-10-28 07:21:28,4116112.0,1,1,4113587,범안동,4116112,범안동,NaN,NaN,111
228,228,621,++CGxI65qxsgBT5BXiMlw3djALPySZXcnMA3AW0UPsw=,41015001,2020-10-23 07:09:04,4196605,2020-10-23 07:11:55,4102119.0,0,1,4196605,소사본동,4102119,범안동,NaN,NaN,112
229,229,622,++CGxI65qxsgBT5BXiMlw3djALPySZXcnMA3AW0UPsw=,41072002,2020-10-23 07:16:04,4113587,2020-10-23 07:20:11,4116112.0,1,1,4113587,범안동,4116112,범안동,NaN,NaN,112


In [22]:
df_change2=df_change2.head(10000)

In [23]:
list_temp=[df_chang2.loc[0,'구분자'],df_chang2.loc[0,'노선 ID'],df_chang2.loc[0,'승차정류장ID'],df_chang2.loc[0,'하차정류장 ID']]
cnt=1
table_temp=[]
for i in range(1,len(df_change2)):
    if (df_change2.loc[i,'구분자']==df_change2.loc[i-1,'구분자']):
        list_temp+=[df_chang2.loc[i,'노선 ID'],df_chang2.loc[i,'승차정류장ID'],df_chang2.loc[i,'하차정류장 ID']]
    else:
        table_temp.append(list_temp)
        list_temp=[df_chang2.loc[i,'구분자'],df_chang2.loc[i,'노선 ID'],df_chang2.loc[i,'승차정류장ID'],df_chang2.loc[i,'하차정류장 ID']]

In [24]:
table_temp

[[0, 102, 1806, 1804, 41030037, 4100939, 4100932],
 [1, 41056004, 4109242, 4100810, 207, 2753, 2758],
 [2, 207, 2758, 2753, 41056004, 4100809, 4109243],
 [3, 41110211, 15366, 8001697, 102, 1804, 1806],
 [4, 102, 1806, 1804, 41110211, 7000168, 15365],
 [5, 207, 2757, 2755, 41015002, 4100290, 4100188],
 [6, 41030014, 4100191, 4101334, 102, 1804, 157],
 [7, 41073005, 4100920, 4101376, 102, 1814, 1006],
 [8, 41261001, 4116009, 4116110, 102, 1804, 210],
 [9, 41261001, 4116009, 4113880, 102, 1804, 210],
 [10, 41261001, 4116009, 4116110, 102, 1804, 210],
 [11, 41261001, 4116009, 4116110, 102, 1804, 210],
 [12, 41261001, 4116009, 4116110, 102, 1804, 210],
 [13, 41261001, 4116009, 4116110, 102, 1804, 210],
 [14, 41261001, 4116009, 4116110, 102, 1804, 210],
 [15, 41261001, 4116009, 4116110, 102, 1804, 210],
 [16, 41071008, 4110865, 4178904, 207, 2758, 2752],
 [17, 114, 4812, 1804, 41030037, 4100939, 4100934],
 [18, 41030003, 4100931, 4100938, 102, 1804, 4812],
 [19, 41056004, 4196537, 4100188, 4

In [5]:
print(df_change2.dtypes)
df_change2=df_change2.astype({'노선 ID':'object','승차정류장ID': 'int64','하차정류장 ID': 'int64'})
df_change2=df_change2.astype({'승차정류장ID': 'object','하차정류장 ID': 'object'})


구분자            int64
노선 ID          int64
승차일시          object
하차일시          object
승차정류장ID        int64
하차정류장 ID     float64
이용객수(다인승)      int64
dtype: object


In [7]:
df_time=df_change2.groupby(["노선 ID","승차정류장ID"]).agg(['size','count']).reset_index()
df_time

노선 ID  승차정류장ID     구분자            승차일시            하차일시          \
                           size   count    size   count    size   count   
0          102     1801   10576   10576   10576   10576   10576   10576   
1          102     1802    3769    3769    3769    3769    3769    3769   
2          102     1803  218046  218046  218046  218046  218046  218046   
3          102     1804  192836  192836  192836  192836  192836  192836   
4          102     1805  185992  185992  185992  185992  185992  185992   
...        ...      ...     ...     ...     ...     ...     ...     ...   
6749  41341002  4178875     478     478     478     478     478     478   
6750  41341002  4178878     185     185     185     185     185     185   
6751  41341002  4178882      14      14      14      14      14      14   
6752  41341002  4196546      55      55      55      55      55      55   
6753  41341002  4196588      40      40      40      40      40      40   

     하차정류장 ID         이용객수(다인승)          
         size   count      size   count  
0       10576   10576     10576   10576  
1        3769    3769      3769    3769  
2      218046  218046    218046  218046  
3      192836  192836    192836  192836  
4      185992  185992    185992  185992  
...       ...     ...       ...     ...  
6749      478     478       478     478  
6750      185     185       185     185  
6751       14      14        14      14  
6752       55      55        55      55  
6753       40      40        40      40  

[6754 rows x 12 columns]

In [ ]:
df_change2

In [ ]:
if 